## Homework 09:  Parallel Programming 02

## Due Date: Apr 19, 2023, 11:59pm

#### Firstname Lastname: Woodward (Buz) Galbraith

#### E-mail: wbg231@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process, say `Process 0`, independently draws $N$ samples uniformly on $[0, 1]$.

How do you explain the results?


**Instructions:** 
Your program should use MPI4PY and collective operations. 
Save your program as 2020_spring_sol09_pr01.py and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr01.py
```


In [13]:
%%writefile 2020_spring_sol09_pr01.py


from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD ## initialize communicator
rank = comm.Get_rank() ## get the rank of each worker. 
X_stats = np.zeros((2)) ## buffer to hold mean and std of our sample
if rank == 0: ## calculate points
    n = 256000    ## size n 
    X = np.random.uniform(low=0,high=1, size=n)  ## X is vector of random draws on uniform zero one of size n 
    # print(X.size)
    # print((X.reshape(16,-1).shape))
    X=X.reshape(16,-1) ## reshape X to be easily broken into chunks for each worker. 
#    X_stats_true = np.array([np.mean(X), np.std(X)])
else:
    X=None ## initialize buffer for non-zero rank workers 
chunk_x = comm.scatter(X, root=0) ## scatter X call it chunk x 
chunk_stats = np.array([np.mean(chunk_x), np.std(chunk_x)]) ## find summary stats for each buffer and store them in vector
# print(rank, chunk_stats) ###

comm.Reduce(chunk_stats, X_stats, MPI.SUM, 0) ## reduce the chunk stats with addition 
X_stats = X_stats/16 ## divide the chunk stats by the number of workers to get the true mean 
if rank == 0: ## terminal case 
    print('final stats of X:\nMean of X = {0}\nStandard Deviation of X = {1}'.format(X_stats[0],X_stats[1])) ## output result
#    print('true x stats =', X_stats_true)

Overwriting 2020_spring_sol09_pr01.py


- We can look at both the numeric logic behind the results and the comp sci logic
- numerically
    - at each chunk we are calculating the mean and standard deviations for list of equal length 
    - given we have these numbers we can effectively think of each as 'sample' and thus take the sample mean, that is add and divide by the length of each sample to get each sample mean, which will thus be equal to the total mean and standard deviation. 
- comp sci
    - it is faster to operate in parallel on smaller chunks of data all at once than run concurrently through one large data structure. 
    - so at rank zero we generate n samples $x_{i}\sim\mathcal{U}(0,1) \quad \forall i\in[1,n]$ then break these samples into 16 vectors (np arrays) of equal size
    - we then check if the rank of the current worker is not equal to zero, in which case we initialize our buffer to none 
    - then we scatter the evenly divided data to each worker. 
    - calculate the mean and standard deviation for each chunk of data
    - use a reduce operation to sum all the means and standard deviations and send the result to rank zero
    - divide that result by 16 to get means in each case 
    - then finally output the total summary statistics. 


---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


**Instructions:** 

Save your program as 2020_sol09_pr02.py; and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr02.py
```



In [16]:
%%writefile 2020_spring_sol09_pr02.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD ## initialize communicator
rank = comm.Get_rank()  ## get the rank of each worker 
num_moments=15 ## save the total number of moments we want to calculate
m_moment = np.zeros((1)) ## buffer to hold each moment as we calculate them   
for m in range(1, num_moments+1): ## goes from the 1st the nth moment. 
    if rank == 0: ##initialize range
        n = 16    ## size n 
        X_range = np.linspace(0, 1000,n*1000)  ## 1 interval between 1 and 1000 of length 16,000
        X_range= X_range.reshape(n,-1) ## the interval subdivided into 16 intervals of length 1,000
    else: 
        X_range=None ## initialize buffer for non-zero rank workers 
    chunk_range = comm.scatter(X_range, root=0) ## scatter the range to all the workers
    dt=chunk_range[1]-chunk_range[0] ## find the width parameter dt
    chunk_int = np.sum(np.power(chunk_range, m) * np.exp(-chunk_range) * dt) ## numerically calculate each chunks integral 
    comm.Reduce(chunk_int,m_moment,op= MPI.SUM,root= 0) ## sum all integrals to get a total integral over the range (ie the moment )
    if rank == 0: ## terminal case 
        print('moment',m ,"=", m_moment[0]) ## output result
## note that we also could have just initialized our buffer as a np array of length num_moments, and added each moment to the ith index
## but we were only told to calculate and output the moments not save them, so this seemed to serve

Overwriting 2020_spring_sol09_pr02.py



**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?

- the ith moments of a Random variable $X\sim e(\lambda)$ is $E[X^i]=\frac{i!}{\lambda^i}$ in our case where $\lambda=1$ we have $E[X^i]=\frac{i!}{1}=i!$
- we can show this with induction 
- consider the base case of the first moment
    - the first moment is given by $E[X^1]=E[X]=\int_{x}xf_{x}(x)dx=\int_{0}^{\infty}xe^{-x}dx=(-x-1)e^{-x}|_{0}^{\infty}$
    - in the limit this can be evaluated as [0+1]=1
- induction assumption: lets assume this holds up to case n-1
- induction step: then for case n we have 
    - $E[X^n]=\int_{x}x^nf_{x}(x)dx=\int_{0}^{\infty}x^n e^{-x}dx=\int_{0}^{\infty}x^n e^{-x}dx=lim_{x\rightarrow \infty} (-x^ne^-x)-(-0^ne^-0)+n\int_{0}^{\infty}x^{n-1} e^{-x}dx=0-0+nE[X^{n-1}]$
    - then by our induction assumption we see the desired result $E[X^n]=\int_{x}x^nf_{x}(x)dx=nE[X^{n-1}]=n((n-1)!)=n!$
- thus by induction this formula will hold. 
- Further note that this is the definition of the gamma function ($\Gamma$) so we could have just used that as well. 
- we can also see that our estimation is correct (accepting a very small amount of error introduced by numeric approximation )